# TRIOS chirp converter

OWCh data is best exported from TRIOS as an excel file, with at least the columns "Stress", "Strain", and "Temperature". This notebook uses `pandas` and `xlrd` to transform the raw `xls` output into a `parquet` file containing only the columns and labels we need for FAST loading and analysis later.

In [ ]:
import pathlib
from ipywidgets import SelectMultiple, interact, Layout

# Location of the data files.
folder = pathlib.Path("data").resolve(strict=True)

print("Selected folder: \n", folder)
all_excel_files = [
    (x.name, x) for x in folder.iterdir() if x.suffix in (".xls", ".xlsx")
]

filenames = []


# This bit of vodoo shows a selection box in the page with all the files in the above folder
# To pick the files, highlight them in any order. Personally I like click-drag selection.
@interact
def populate_filenames(
    x=SelectMultiple(
        options=all_excel_files,
        index=tuple(
            range(len(all_excel_files))
        ),  # this magick selects all available files by default
        description="Available files",
        layout=Layout(width="auto"),
        rows=20,  # number of rows visible in the box
    )
):
    global filenames
    filenames = sorted(x)

In [ ]:
import pandas as pd
import numpy as np

idx = pd.IndexSlice

assert filenames
for filename in filenames:
    print(filename)
    # we must not parse the "details" sheet, so we cannot use the simpler pd.read_excel
    xlfile = pd.ExcelFile(filename)
    waves_df = (
        pd.concat(
            xlfile.parse(
                # Parse only sheets with names given to our chirp data steps
                [
                    name
                    for name in xlfile.sheet_names
                    if name.startswith("Arbitrary Wave")
                ],
                header=[0, 1, 2],
                dtype="f4",
            ).values(),
            axis="columns",
            join="inner",  # rarely, sheets can have uneven rows
        )
        .loc[
            pd.IndexSlice[:],
            pd.IndexSlice[:, ("Strain", "Stress", "Temperature", "Time"), :],
        ]
        .rename(columns=lambda n: n.split()[-1], level=0)
    )
    assert not np.any(np.isnan(waves_df.values))
    waves_df.columns = waves_df.columns.droplevel(2)
    display(waves_df.describe())
    waves_df.to_parquet(
        filename.with_suffix(".parquetzstd"),
        compression="zstd",
    )